## Batch withdrawals tests

In [1]:
from docs.scenarios.setup import *

accounts = setup()
web3, etherlink_account, tezos_account = accounts
_, ticketer, erc20, token_bridge_helper, _ = load_contracts(*accounts)

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `209.948680 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `503.089610897 ꜩ`


### Deploy BulkWithdrawal:
- TODO: rename to the BatchWithdrawal
- This is the contract that implements `xtz_withdrawal_precompile` and `fa_withdraw_precompile` interfaces.
- When user calls these precompile entrypoints, it make `N` copies of this call to the corresponding withdrawal precompile.
- Number of the calls `N` provided during origination.

In [27]:
bulk_withdrawal = BulkWithdrawalHelper.originate_from_file(
    web3=web3,
    account=etherlink_account,
    filename=make_filename('BulkWithdrawal'),
    constructor_args=(
        FA_WITHDRAWAL_PRECOMPILE,
        XTZ_WITHDRAWAL_PRECOMPILE,
        100,
    )
)
bulk_withdrawal.address

'0x865e2887baFE9BDD5E68d0cc5Fb9eEB9dCE535Fa'

### Or load already deployed contract:

In [2]:
bulk_withdrawal = BulkWithdrawalHelper.from_address(web3, etherlink_account, '0x2A0030903E5313bf145d8631E87aAc5237F9aD26')
bulk_withdrawal.address

'0x2A0030903E5313bf145d8631E87aAc5237F9aD26'

### FA Token deposit:
- It is required to deposit FA token to the contract so it would be able to execute withdrawals.

In [29]:
result = deposit.callback(
    token_bridge_helper_address=token_bridge_helper.address,
    amount=1_000,
    receiver_address=bulk_withdrawal.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL
)

Making deposit using Helper `KT1JLZe4qTa76y6Us2aDoRNUgZyssSDUr6F5`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - Ticketer: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
  - Deposit params:
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
      * Receiver address: `0x865e2887baFE9BDD5E68d0cc5Fb9eEB9dCE535Fa`
      * Amount: `1_000`
Successfully executed Deposit, tx hash: `ongjWFXToqbYUFNDaFjujNoPWBPDxDx5XHgfhZQtWGYseipdxeq`


### FA Token withdrawal:
- The withdraw CLI used with `batch_withdrawal.address` provided as withdrawal precompile contract.
- The `BatchWithdrawal` then executes `fa_withdrawal_precompile` multiple times spamming the same transaction.

In [32]:
withdraw.callback(
    erc20_proxy_address=erc20.address,
    tezos_side_router_address=ticketer.address,
    amount=1,
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=bulk_withdrawal.address,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0xf68997eCC03751cb99B5B36712B213f11342452b`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Ticket owner: `0xf68997eCC03751cb99B5B36712B213f11342452b`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601c0137ec3632f70c119ea958dca84cdaafa42024000`
      * Amount: `1`
      * Ticketer address bytes: `0x01c0137ec3632f70c119ea958dca84cdaafa42024000`
      * Content bytes: `0x0707000005090a000000cc0502000000c607040100000010636f6e74726163745f616464726573730a000000244b54315632616b314d664e643377346f794b44363465685955374b34437270556344475207040100000008646563696d616c730a0000000136070401000000046e616d650a0000001454657374205465746865722055534420763134310704010000000673796d626f6c0a0000000d544553545f555344745f3134310704010000000874

'0x9645796b5153072f94c2c8390641da9603471eba0588408a5d414e9603f1754f'

### Native deposit:
- To withdraw XTZ from the contract, it is required to deposit some.

In [33]:
opg_hash = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    receiver_address=bulk_withdrawal.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1VEjeQfDBSfpDH5WeBM5LukHPGM2htYEh3`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
      * Receiver address: `0x2A0030903E5313bf145d8631E87aAc5237F9aD26`
      * Amount (mutez): `1_000_000`
Successfully executed XTZ deposit, tx hash: `oo7GRLS95sGVLni2ZXARC3Uwdnsgoc61mWiY5GT9YCCiZMbUWss`


### Native withdrawal:
- The xtz withdraw CLI used with `batch_withdrawal.address` provided as withdrawal precompile contract.
- The `BatchWithdrawal` then executes `xtz_withdrawal_precompile` multiple times spamming the same transaction.

In [35]:
opg_hash = xtz_withdraw.callback(
    amount=10**12,
    xtz_withdraw_precompile=bulk_withdrawal.address,
    receiver_address='tz1burnburnburnburnburnburnburjAYjjX',
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL
)

Making XTZ withdrawal from `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Receiver: `tz1burnburnburnburnburnburnburjAYjjX`
      * Amount (wei):: `1_000_000_000_000`
      * Amount (mutez):: `1`
Successfully initiated XTZ withdrawal, tx hash: `0x3a116245e9cb547dc101653d6e7014abf663d67ec378500291d77b5d2684d163`
